In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ExampleApp") \
        .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/26 14:07:57 WARN Utils: Your hostname, epsilon, resolves to a loopback address: 127.0.1.1; using 192.168.9.119 instead (on interface wlp2s0f0)
25/11/26 14:07:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/26 14:07:57 WARN Utils: Your hostname, epsilon, resolves to a loopback address: 127.0.1.1; using 192.168.9.119 instead (on interface wlp2s0f0)
25/11/26 14:07:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/26 14:07:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spar

In [4]:
# Load dataset
df = spark.read.csv("DataSet_Emails.csv", 
                    header=True, 
                    inferSchema=True, 
                    multiLine=True,  
                    escape='"'      
                    )
print("Dataset loaded successfully!")
df.show(5, truncate=50)

Dataset loaded successfully!
+---+----------+--------------------------------------------------+-----+----------+--------------------------------------------------+--------------------------------------------------+----------+
|_c0|message_id|                                              text|label|label_text|                                           subject|                                           message|      date|
+---+----------+--------------------------------------------------+-----+----------+--------------------------------------------------+--------------------------------------------------+----------+
|  0|     33214|any software just for 15 $ - 99 $ understanding...|    1|      spam|                 any software just for 15 $ - 99 $|understanding oem software\nlead me not into te...|2005-06-18|
|  1|     11929|perspective on ferc regulatory action client co...|    0|       ham|perspective on ferc regulatory action client co...|19 th , 2 : 00 pm edt\nperspective on ferc r

25/11/26 14:08:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , message_id, text, label, label_text, subject, message, date
 Schema: _c0, message_id, text, label, label_text, subject, message, date
Expected: _c0 but found: 
CSV file: file:///home/epsilon/Desktop/folder_0/projects/sprint_3/Anti-Spam-Intelligent/DataSet_Emails.csv


In [5]:
# 1. EXAMINE DATASET STRUCTURE
print("=" * 60)
print("DATASET STRUCTURE")
print("=" * 60)

# Display schema
print("\nColumn Types:")
df.printSchema()

# Dataset size
print(f"\nTotal rows: {df.count()}")
print(f"Total columns: {len(df.columns)}")

# Show sample data
print("\nSample emails:")
df.show(3, truncate=100)

DATASET STRUCTURE

Column Types:
root
 |-- _c0: integer (nullable = true)
 |-- message_id: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- label: integer (nullable = true)
 |-- label_text: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- message: string (nullable = true)
 |-- date: date (nullable = true)



25/11/26 14:08:04 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , message_id, text, label, label_text, subject, message, date
 Schema: _c0, message_id, text, label, label_text, subject, message, date
Expected: _c0 but found: 
CSV file: file:///home/epsilon/Desktop/folder_0/projects/sprint_3/Anti-Spam-Intelligent/DataSet_Emails.csv



Total rows: 31716
Total columns: 8

Sample emails:
+---+----------+----------------------------------------------------------------------------------------------------+-----+----------+---------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------+
|_c0|message_id|                                                                                                text|label|label_text|                                                              subject|                                                                                             message|      date|
+---+----------+----------------------------------------------------------------------------------------------------+-----+----------+---------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+---------

25/11/26 14:08:04 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , message_id, text, label, label_text, subject, message, date
 Schema: _c0, message_id, text, label, label_text, subject, message, date
Expected: _c0 but found: 
CSV file: file:///home/epsilon/Desktop/folder_0/projects/sprint_3/Anti-Spam-Intelligent/DataSet_Emails.csv


In [6]:
# 2. DETECT AND HANDLE MISSING VALUES
from pyspark.sql.functions import col , udf

print("=" * 60)
print("MISSING VALUES ANALYSIS")
print("=" * 60)

for column in df.columns:
    null_count = df.filter(col(column).isNull()).count()
    print(f"{column}: {null_count} missing values")

df_clean = df.dropna()
print(f"\nRows after removing missing values: {df_clean.count()}")

MISSING VALUES ANALYSIS


25/11/26 14:08:08 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , message_id, text, label, label_text, subject, message, date
 Schema: _c0, message_id, text, label, label_text, subject, message, date
Expected: _c0 but found: 
CSV file: file:///home/epsilon/Desktop/folder_0/projects/sprint_3/Anti-Spam-Intelligent/DataSet_Emails.csv


_c0: 0 missing values


25/11/26 14:08:08 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , message_id, text, label, label_text, subject, message, date
 Schema: _c0, message_id, text, label, label_text, subject, message, date
Expected: _c0 but found: 
CSV file: file:///home/epsilon/Desktop/folder_0/projects/sprint_3/Anti-Spam-Intelligent/DataSet_Emails.csv


message_id: 0 missing values


...

message: 345 missing values


25/11/26 14:08:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , message_id, text, label, label_text, subject, message, date
 Schema: _c0, message_id, text, label, label_text, subject, message, date
Expected: _c0 but found: 
CSV file: file:///home/epsilon/Desktop/folder_0/projects/sprint_3/Anti-Spam-Intelligent/DataSet_Emails.csv


date: 0 missing values


25/11/26 14:08:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , message_id, text, label, label_text, subject, message, date
 Schema: _c0, message_id, text, label, label_text, subject, message, date
Expected: _c0 but found: 
CSV file: file:///home/epsilon/Desktop/folder_0/projects/sprint_3/Anti-Spam-Intelligent/DataSet_Emails.csv



Rows after removing missing values: 31148


In [31]:
# 3. DETECT AND HANDLE DUPLICATES
print("=" * 60)
print("DUPLICATES ANALYSIS")
print("=" * 60)

total_rows = df_clean.count()
distinct_rows = df_clean.distinct().count()
duplicates = total_rows - distinct_rows

print(f"Total rows: {total_rows}")
print(f"Distinct rows: {distinct_rows}")
print(f"Duplicates: {duplicates}")

# Remove duplicates
df_clean = df_clean.dropDuplicates()
print(f"\nRows after removing duplicates: {df_clean.count()}")

df.columns

DUPLICATES ANALYSIS


25/11/26 12:04:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , message_id, text, label, label_text, subject, message, date
 Schema: _c0, message_id, text, label, label_text, subject, message, date
Expected: _c0 but found: 
CSV file: file:///home/epsilon/Desktop/folder_0/projects/sprint_3/Anti-Spam-Intelligent/DataSet_Emails.csv
25/11/26 12:04:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , message_id, text, label, label_text, subject, message, date
 Schema: _c0, message_id, text, label, label_text, subject, message, date
Expected: _c0 but found: 
CSV file: file:///home/epsilon/Desktop/folder_0/projects/sprint_3/Anti-Spam-Intelligent/DataSet_Emails.csv
25/11/26 12:04:16 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , message_id, text, label, label_text, subject, message, date
 Schema: _c0, message_id, text, label, label_text, subject, message, date
Expected: _c0 but found: 
CSV file: file:///hom

Total rows: 31148
Distinct rows: 31148
Duplicates: 0

Rows after removing duplicates: 31148


['_c0',
 'message_id',
 'text',
 'label',
 'label_text',
 'subject',
 'message',
 'date']

In [32]:
# 4. ANALYZE CLASS DISTRIBUTION
print("=" * 60)
print("CLASS DISTRIBUTION ANALYSIS")
print("=" * 60)

# Count spam vs ham
spam_count = df_clean.filter(col('label_text') == "spam").count()
ham_count = df_clean.filter(col('label_text') == "ham").count()
total = df_clean.count()

print(f"\nSpam emails: {spam_count} ({spam_count/total*100:.2f}%)")
print(f"Ham emails: {ham_count} ({ham_count/total*100:.2f}%)")
print(f"Total emails: {total}")

# Check balance
if spam_count > ham_count:
    ratio = spam_count / ham_count
else:
    ratio = ham_count / spam_count
    
print(f"\nClass imbalance ratio: 1:{ratio:.2f}")
if ratio > 2:
    print("⚠ Dataset is imbalanced - consider using techniques like SMOTE or class weights")

CLASS DISTRIBUTION ANALYSIS


25/11/26 12:04:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , message_id, text, label, label_text, subject, message, date
 Schema: _c0, message_id, text, label, label_text, subject, message, date
Expected: _c0 but found: 
CSV file: file:///home/epsilon/Desktop/folder_0/projects/sprint_3/Anti-Spam-Intelligent/DataSet_Emails.csv
25/11/26 12:04:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , message_id, text, label, label_text, subject, message, date
 Schema: _c0, message_id, text, label, label_text, subject, message, date
Expected: _c0 but found: 
CSV file: file:///home/epsilon/Desktop/folder_0/projects/sprint_3/Anti-Spam-Intelligent/DataSet_Emails.csv
25/11/26 12:04:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , message_id, text, label, label_text, subject, message, date
 Schema: _c0, message_id, text, label, label_text, subject, message, date
Expected: _c0 but found: 
CSV file: file:///hom


Spam emails: 15645 (50.23%)
Ham emails: 15503 (49.77%)
Total emails: 31148

Class imbalance ratio: 1:1.01


In [33]:
from pyspark.sql.functions import lower, regexp_replace, concat_ws

text_column = [col for col in df_clean.columns if col != 'label_text'][0]

spam_text = df_clean.filter(col('label_text') == 'spam').select(text_column).rdd.map(lambda x: x[0]).collect()
spam_text_combined = ' '.join([str(text) for text in spam_text if text])

ham_text = df_clean.filter(col('label_text') == 'ham').select(text_column).rdd.map(lambda x: x[0]).collect()
ham_text_combined = ' '.join([str(text) for text in ham_text if text])

print(f"Spam text collected: {len(spam_text_combined)} characters")
print(f"Ham text collected: {len(ham_text_combined)} characters")

25/11/26 12:04:19 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , message_id, text, label, label_text, subject, message, date
 Schema: _c0, message_id, text, label, label_text, subject, message, date
Expected: _c0 but found: 
CSV file: file:///home/epsilon/Desktop/folder_0/projects/sprint_3/Anti-Spam-Intelligent/DataSet_Emails.csv
25/11/26 12:04:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , message_id, text, label, label_text, subject, message, date
 Schema: _c0, message_id, text, label, label_text, subject, message, date
Expected: _c0 but found: 
CSV file: file:///home/epsilon/Desktop/folder_0/projects/sprint_3/Anti-Spam-Intelligent/DataSet_Emails.csv


Spam text collected: 88474 characters
Ham text collected: 87469 characters


In [34]:
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

normalize_word = WordNetLemmatizer()

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')

def preprocess_text(text):
    text = text.lower()
    text = regexp_replace(text, r'http\S+|www\S+|https\S+', 'url')
    text = regexp_replace(text, r'\d+', 'number')
    text = regexp_replace(text, r'[^\w\s]', '')
    text = text.lower()
    
    stop_words = set(stopwords.words("english"))

    tokens = word_tokenize(text)

    normalized_word = [normalize_word.lemmatize(token) for token in tokens if token not in stop_words]
    
    return normalize_word

vectorize = TfidfVectorizer(tokenizer=preprocess_text, token_pattern=None)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/epsilon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/epsilon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/epsilon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/epsilon/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [35]:
from pyspark.sql.types import ArrayType, StringType

clean_udf = udf(preprocess_text, ArrayType(StringType()))
new_df = df_clean.withColumn("tokens", clean_udf(col("text")))

In [15]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from pyspark.sql.functions import udf, col, concat_ws, coalesce, lit
from pyspark.sql.types import ArrayType, StringType
import re

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    if text is None or text == "":
        return []
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', 'url', text)
    text = re.sub(r'\d+', 'number', text)
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    normalized_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and token.strip()]
    return normalized_tokens

clean_udf = udf(preprocess_text, ArrayType(StringType()))

cleaned_df = df_clean.withColumn(
    "all_text",
    concat_ws(" ", coalesce(col("text"), lit("")), coalesce(col("subject"), lit("")), coalesce(col("message"), lit("")))
).withColumn("tokens", clean_udf(col("all_text"))).select("label", "label_text", "tokens")

cleaned_df.show(5, truncate=False)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/epsilon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/epsilon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/epsilon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/epsilon/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/epsilon/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
25/11/26 14:16:01 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , message_id, text, label, label_text, subject, message, date
 Schema: _c0, message_id, text, label, label_text, subject, message, date
Expected: _c0 but found: 
CSV file: file:///home/epsilon/Desktop/folder_0/projects/sprint_3/Anti-Spam-Intelligent/DataSe

+-----+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
new_df.columns

['_c0',
 'message_id',
 'text',
 'label',
 'label_text',
 'subject',
 'message',
 'date',
 'combined_text',
 'text_clean',
 'tokens']